In [14]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [21]:
!pip install torch
!pip install torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [22]:
# 訓練データをdatasetsからダウンロード
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# テストデータをdatasetsからダウンロード
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [23]:
batch_size = 64

# データローダの作成
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [24]:
# 訓練に際して，可能であればGPU(cuda)を設定します．GPUが搭載されていない場合はCPUを使用します．
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# モデルの定義
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # 28*28=784
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10), # 10はクラス数
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) # GPUを使用する場合は，データをGPUに転送します．

        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [28]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval() # モデルを評価モードに設定 
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [30]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 1.497800 [    0/60000]
loss: 1.519281 [ 6400/60000]
loss: 1.282117 [12800/60000]
loss: 1.463509 [19200/60000]
loss: 1.400621 [25600/60000]
loss: 1.475545 [32000/60000]
loss: 1.470335 [38400/60000]
loss: 1.373417 [44800/60000]
loss: 1.575682 [51200/60000]
loss: 1.648710 [57600/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 0.022539 

Epoch 2
-------------------------------
loss: 1.439084 [    0/60000]
loss: 1.467010 [ 6400/60000]
loss: 1.244010 [12800/60000]
loss: 1.437488 [19200/60000]
loss: 1.353842 [25600/60000]
loss: 1.446131 [32000/60000]
loss: 1.429904 [38400/60000]
loss: 1.341710 [44800/60000]
loss: 1.535153 [51200/60000]
loss: 1.615013 [57600/60000]
Test Error: 
 Accuracy: 55.5%, Avg loss: 0.022013 

Epoch 3
-------------------------------
loss: 1.394619 [    0/60000]
loss: 1.425918 [ 6400/60000]
loss: 1.214410 [12800/60000]
loss: 1.419011 [19200/60000]
loss: 1.321972 [25600/60000]
loss: 1.422214 [32000/60000]
loss: 1.401146 [38400/

In [32]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [33]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle Boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle Boot", Actual: "Ankle Boot"
